In [1]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

Mounted at /content/drive


In [2]:
flight = pd.read_excel('/content/drive/My Drive/FYP2/Data_FlightRaw.xlsx')

In [3]:
flightm = flight.drop_duplicates(subset=['airline_iata', 'flight_number'])
flightm = flightm.dropna(subset=['flight_iata'])
flightm = flightm[['airline_iata','flight_number','flight_iata']].reset_index(drop=True)
flightm.head()

,airline_iata,flight_number,flight_iata
0,MH,194,MH194
1,AK,6440,AK6440
2,MH,196,MH196
3,TR,469,TR469
4,SQ,297,SQ297


In [6]:
#API KEY EXPIRED need to insert new API KEY

import requests
import json

url = "https://api.flightstats.com/flex/flightstatus/rest/v2/json/flight/status/"
app_id = "95fdb9ed"
app_key = "88a86a1360e6b3647ce9fc545552982a"

model =[]
turboProp = []
widebody =[]

for index, row in flightm.iterrows():
  carrier = str(row['airline_iata'])
  flight_number = str(row['flight_number'])

  year = "2023"
  month = 9
  day = 7

  for i in range (7):
    print(str(row['flight_iata'])+' : '+str(month) +" / " + str(day))
    # Make the API request with the parameters
    response = requests.get(
        url + carrier + "/" + flight_number + "/arr/" + year + "/0" + str(month) + "/" + str(day),
        params={
            "appId": app_id,
            "appKey": app_key,
            "utc": "false"
        }
    )

    # Parse the JSON response content
    json_data = json.loads(response.content)
    if (json_data['appendix']['equipments'] != []):
      model.append(json_data['appendix']['equipments'][0]['name'])
      turboProp.append(json_data['appendix']['equipments'][0]['turboProp'])
      widebody.append(json_data['appendix']['equipments'][0]['widebody'])
      break
    else:
      if i == 6 :
        model.append("None")
        turboProp.append("None")
        widebody.append("None")
      else:
        day+=1
        if day == 32:
          day = 1
          month +=1
        continue


flightm['Aircraft_Model'] = model
flightm['Aircraft_isTurboProp'] = turboProp
flightm['Aircraft_isWidebody'] = widebody
flightm = flightm[['flight_iata','Aircraft_Model','Aircraft_isTurboProp','Aircraft_isWidebody']]
flightm

MH194 : 9 / 7


KeyError: ignored

In [ ]:
from google.colab import files
flightm.to_csv('flightmodel.csv', encoding = 'utf-8-sig',index=False)
files.download('flightmodel.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>